In [1]:
from ipynb.fs.defs.functions import *

from datetime import datetime
import networkx as nx
import math
import numpy as np
import random
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite
import sys
import contextlib
import itertools
from math import comb
from numpy import linalg as LA
import math
from numpy.linalg import matrix_power
import matplotlib.pyplot as plt
from math import factorial as f
import networkx as nx
import itertools
import time
import pandas as pd
import random as rnd

def theoretic_fixation_moran(nodes,fitness):
    if fitness==1:
        return 1/(nodes)
    else:
        k=(1-1/fitness)/(1-1/(pow(fitness,nodes)))
        return k

In [56]:
    ### Generate the 100 connected random graphs
    ### Each of 100 graph will have 100 random weighted versions
    start_time = datetime.now()
    random_graph_list=[]
    c=0
    while len(random_graph_list)<100:
        c=c+1
        p=(c%10)/10
        ##we are varying p as we go along
    
        G=nx.fast_gnp_random_graph(10, p, seed=None, directed=False)
        if nx.is_connected(G) == False :
            continue
        else:
            random_graph_list.append(G)
        

        
    ### Generating the weighted versions:
    ### Each row has 101 columns, 1 for the original network, 100 for the weighted networks.
    ### random_graph_data will store the fixation probability of each of the networks
    random_graph_data= np.zeros((100,101), dtype=object)
    random_graph_collection =  np.zeros((100,101), dtype=object)

    for i in range(len(random_graph_list)):#for each network
    
        random_graph_collection[i,0]=random_graph_list[i]
        #random_graph_data[i,0]=[matrix_solver(T_weightMat(random_graph_collection[i,0],k/10))-theoretic_fixation_moran(10,k/10) for  k in range(1,20)]
        for j in range(1,101): #for each weighted analogue of a network
            G=random_graph_collection[i,0]
            for u,v in G.edges():
                G[u][v]['weight']=rnd.random()

            random_graph_collection[i,j]=G
            
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:00.382121


In [51]:
def random_10_nodes_weighted(index,weight_index):    
    return [matrix_solver(T_weightMat(random_graph_collection[index,weight_index],t/10))-theoretic_fixation_moran(10,t/10) for  t in range(1,20)]



In [35]:
np.save('random_graph_collection.npy', random_graph_collection)
np.save('random_graph_data.npy', random_graph_data )

SyntaxError: unexpected EOF while parsing (<ipython-input-35-7bf25d82f5e0>, line 1)

In [ ]:
import time
from timeit import default_timer as timer
from multiprocessing import Pool, cpu_count


def main(j):
    
    start = timer()
    c=[]
    arg=[]

    print(f'starting computations on {cpu_count()} cores')
    
    for l in range(101):
        arg.append((j,l))


    with Pool() as pool:
        c.append(pool.starmap(random_10_nodes_weighted, arg))
        

    end = timer()
    print(f'elapsed time: {end - start}')
    return c

if __name__ == '__main__':
    random_graph_data =  np.zeros((100,101), dtype=object)
    c=[]
    for k in range(100):
        random_graph_data[k]=main(k)
        np.save('random_graph_data.npy', random_graph_data )
    
        


starting computations on 56 cores
elapsed time: 4829.829804899171
starting computations on 56 cores
elapsed time: 4814.532375191804
starting computations on 56 cores
elapsed time: 4797.140383997001
starting computations on 56 cores
elapsed time: 5072.240246076835
starting computations on 56 cores
elapsed time: 4857.937398198992
starting computations on 56 cores
elapsed time: 4801.409766741097
starting computations on 56 cores


In [36]:
random_graph_data[22][13][0]

[-6.26488796415371e-11,
 -2.2624553628106084e-08,
 -5.980428212773214e-07,
 -5.263875160013991e-06,
 -2.4395619257227934e-05,
 -7.153911247797639e-05,
 -0.0001426408184045068,
 -0.00019354601722283704,
 -0.00015711977602071714,
 1.3877787807814457e-17,
 0.00024867903118278023,
 0.0005272164069345664,
 0.0007833091824055449,
 0.0009901994503546674,
 0.001141950971501382,
 0.0012437389750514227,
 0.0013047127830048688,
 0.0013342654939783594,
 0.0013405377609437275]

## Classification

In [2]:
 random_graph_data= np.load('random_graph_data.npy', allow_pickle=True)

In [22]:
len(random_graph_data[49])

101

In [37]:
##classifying 

classifier = np.zeros((100, 101), dtype=object)

check=[]

for i in range(50):
    for j in range(1,100):
        for k in range(3,19):
            if k==9:
                continue
            check.append(np.sign(random_graph_data[i][j][1][k]))
        #print(check)
        #print()
        #print(i,j,k)
        if check==[1,1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1]:
            #suppressor
            classifier[i,j]= "Suppressor"
        elif check==[-1,-1,-1,-1,-1,-1,1,1,1,1,1,1,1,1,1]:
            #amplifier
            classifier[i,j]= "Amplifier"
        elif check==[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]:
            #isothermal
            classifier[i,j]= "Isothermal"
        else:
            #neither
            classifier[i,j]= "Neither"
        check=[]
            

In [38]:
###list of suppressors and 'neither' networks
amp=0
sup=0
neither=0
iso=0

sup_list=[]
neither_list=[]
iso_list=[]

for i in range(50):
    for j in range(101):
        if classifier[i,j]== "Suppressor":
            sup+=1
            sup_list.append([i,j])
        elif classifier[i,j]== "Amplifier":
            amp+=1
        elif classifier[i,j]== "Isothermal":
            iso+=1
            iso_list.append([i,j])
        else:
            neither+=1
            neither_list.append([i,j])
            
            
print(amp, sup, neither,iso)

2970 1287 793 0


In [179]:
dfdata=[]
s=0
a=0
n=0
for i in range(112):
    #each row will contain 4 things, graph enumeration, number of amps, sups, neither 
    a= list(classifier[i]).count('Amplifier')#count number of occurrences of amplifier in classifier
    s= list(classifier[i]).count('Suppressor')
    n= 10-a-s
    dfdata.append([i,a,s,n])

In [181]:
#reading in the dataframe
df.to_pickle('weighted_networks.pkl')  
df = pd.read_pickle('weighted_networks.pkl')

In [1]:
df.head(100)

NameError: name 'df' is not defined

In [208]:
df.to_csv('Weighted_Networks.csv')

In [202]:
network_type[102]="Suprressor"